In [1]:
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now
from tinkoff.invest.schemas import InstrumentStatus
from tinkoff.invest.constants import INVEST_GRPC_API

In [2]:
%%capture
# Load secrets

load_dotenv("../secrets.env")

token_ro = os.getenv("TINKOFF_TOKEN_RO")

In [3]:
imoex_shares_path = "../data/tickers_info/imoex_shares.csv"
imoex_index_shares_path = "../data/tickers_info/imoex_index_shares.csv"
imoex_index_tickers_path = "../data/tickers_info/imoex_index_tickers.csv"
figi_load_path = "../data/historical/figi_to_load.txt"

In [4]:
_client = Client(token_ro, target=INVEST_GRPC_API)
client = _client.__enter__()

In [5]:
# My accounts:

client.users.get_accounts()

GetAccountsResponse(accounts=[Account(id='2040067759', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='Брокерский счёт', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2020, 8, 24, 0, 0, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_READ_ONLY: 2>)])

In [6]:
res = client.instruments.shares(instrument_status=InstrumentStatus.INSTRUMENT_STATUS_BASE)
traded_shares = pd.DataFrame(res.instruments)
exchanges = sorted(traded_shares["exchange"].unique())
print(f"Number of shares per exchange: {traded_shares.groupby('exchange').exchange.count()}")

Number of shares per exchange: exchange
LSE_MORNING              11
MOEX                     75
MOEX_EVENING_WEEKEND     46
MOEX_PLUS                11
MOEX_WEEKEND              7
SPB                     866
SPB_HK                  115
SPB_MORNING             881
SPB_RU_MORNING            1
spb_cis                   1
spb_close                 5
spb_etf_liquidity         3
spb_til                   1
Name: exchange, dtype: int64


In [7]:
# Trading schedules

for exchange in exchanges:
    times = client.instruments.trading_schedules(exchange=exchange).exchanges[0]
    t_open = times.days[1].start_time.strftime("%H:%M:%S")
    t_close = times.days[1].end_time.strftime("%H:%M:%S")
    print(f"{times.exchange} : {t_open} - {t_close}")

LSE_MORNING : 07:00:00 - 15:40:00
MOEX : 07:00:00 - 15:39:59
MOEX_EVENING_WEEKEND : 07:00:00 - 20:49:59
MOEX_PLUS : 07:00:00 - 20:50:00
MOEX_WEEKEND : 07:00:00 - 15:39:59
SPB : 11:30:00 - 22:45:00
SPB_HK : 00:00:00 - 10:00:00
SPB_MORNING : 07:00:00 - 22:45:00
SPB_RU_MORNING : 07:00:00 - 21:00:00
SPB_CIS : 05:00:00 - 15:00:00
SPB_CLOSE : 21:00:00 - 20:59:59
SPB_ETF_LIQUIDITY : 13:30:00 - 20:00:00
SPB_TIL : 13:30:00 - 22:45:00


In [8]:
# Example of a stock

traded_shares.loc[traded_shares["ticker"] == "SBER"].squeeze()

figi                                                      BBG004730N88
ticker                                                            SBER
class_code                                                        TQBR
isin                                                      RU0009029540
lot                                                                 10
currency                                                           rub
klong                                          {'units': 2, 'nano': 0}
kshort                                         {'units': 2, 'nano': 0}
dlong                                  {'units': 0, 'nano': 200000000}
dshort                                 {'units': 0, 'nano': 199900000}
dlong_min                              {'units': 0, 'nano': 105600000}
dshort_min                              {'units': 0, 'nano': 95400000}
short_enabled_flag                                                True
name                                                         Сбер Банк
exchan

In [9]:
# IMOEX shares

tickers = pd.read_csv(imoex_index_tickers_path).squeeze()
tickers

0      AFKS
1      AFLT
2      AGRO
3      ALRS
4      CBOM
5      CHMF
6      ENPG
7      FIVE
8      FIXP
9      GAZP
10     GLTR
11     GMKN
12     HYDR
13     IRAO
14     LKOH
15     MAGN
16     MGNT
17     MOEX
18     MTSS
19     NLMK
20     NVTK
21     OZON
22     PHOR
23     PIKK
24     PLZL
25     POLY
26     ROSN
27     RTKM
28     RUAL
29     SBER
30    SBERP
31     SNGS
32    SNGSP
33     TATN
34    TATNP
35     TCSG
36    TRNFP
37     VKCO
38     VTBR
39     YNDX
Name: IMOEX shares, dtype: object

In [10]:
imoex_shares = traded_shares.loc[traded_shares["exchange"].str.contains("MOEX"), ["figi", "ticker", "name", "uid"]].set_index("figi")
imoex_shares.to_csv(imoex_shares_path)
imoex_index_shares = imoex_shares.loc[imoex_shares["ticker"].isin(tickers)]
imoex_index_shares.to_csv(imoex_index_shares_path)
imoex_index_shares

,ticker,name,uid
figi,,,
BBG000RMWQD4,ENPG,En+ Group,e2bd2eba-75de-4127-b39c-2f2dbe3866c3
BBG004PYF2N3,POLY,Polymetal,127361c2-32ec-448c-b3ec-602166f537ea
BBG004RVFCY3,MGNT,Магнит,ca845f68-6c43-44bc-b584-330d2a1e5eb7
BBG00ZHCX1X2,FIXP,Fix Price Group,ba64a3c7-dd1d-4f19-8758-94aac17d971b
BBG00178PGX3,VKCO,VK,3c0748ce-9b49-43e9-b788-048a6cb65174
BBG004S689R0,PHOR,ФосАгро,9978b56f-782a-4a80-a4b1-a48cbecfd194
BBG007N0Z367,AGRO,РусАгро,46ae47ee-f409-4776-bf20-43a040b9e7fb
BBG004S68829,TATNP,Татнефть - привилегированные акции,efdb54d3-2f92-44da-b7a3-8849e96039f6
BBG004S68507,MAGN,Магнитогорский металлургический комбинат,7132b1c9-ee26-4464-b5b5-1046264b61d9


In [17]:
with open(figi_load_path, "w") as f:
    for figi in imoex_index_shares.index:
        f.write(f"{figi}\n")

In [5]:
_client.__exit__(None, None, None)

False